# Python API examples


## Creates

- createUser
- createMarket
- createTrade

### Create user

Creates a new user. Returns signature and verify keys.

~~~~
import requests
url = 'https://blocmarket.herokuapp.com/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)
~~~~

Example return as json:

~~~~
{'signingKey': '0cca0a3739eba76cc78823d9f6e898379014d8c53172f5e45c171a3c54a9f477',
 'traderId': '1',
 'verifyKey': 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'}
~~~~
        
### Create/update market

Creates/updates a market. Returns True and market details if successful. 

Specify:
- Market root and branch id
- Minimum and maximum outcomes
- Trader Id
- Sign and verify keys

Note that sending the private key is a terrible idea. In production signing is done on client side.

~~~~
sk = '0cca0a3739eba76cc78823d9f6e898379014d8c53172f5e45c171a3c54a9f477'
vk = 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'
tId = 2

url = 'https://blocmarket.herokuapp.com/createMarket'
headers = {'content-type': 'application/json'}
content_makemarket = {"signingKey": sk,
                    "traderId": tId, 
                    "verifyKey": vk,
                    "marketRootId": 1, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1e4}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
~~~~

Example return as json:

~~~~
{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '1', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': False, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 1,
 'marketId': 1,
 'marketMax': 1e4,
 'marketMin': 0,
 'marketRootId': 1,
 'traderId': 2}

~~~~
    
### Create trade

Create a trade. Returns True and trade details if successful.

Specify:

- Trader Id
- Market Id
- Price
- Quantity
- Sign and verify keys

~~~~
url = 'https://blocmarket.herokuapp.com/createTrade'
headers = {'content-type': 'application/json'}
content_maketrade = {"signingKey": sk,
                     "traderId": int(tId),
                     "verifyKey": vk,
                     "marketId": mkId,
                     "price": 5510,
                     "quantity":1}
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)



~~~~

Example return as json:
~~~~

{'allChecks': "{'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 5510,
 'quantity': 1,
 'traderId': 1}


~~~~

## Views

- viewMarketBounds
- viewOrderBook
- viewOpenTrades
- viewMatchedTrades

### View markets

~~~~
url = 'https://blocmarket.herokuapp.com/viewMarketBounds'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":2,"2":3,"3":4},
"marketRootId":{"0":1,"1":2,"2":3,"3":4},
"marketBranchId":{"0":1,"1":1,"2":1,"3":1},
"marketMin":{"0":0.0,"1":0.0,"2":0.0,"3":0.0},
"marketMax":{"0":,"1":7000,"2":7000,"3":7000}}'
~~~~

### View full order book

~~~~
url = 'https://blocmarket.herokuapp.com/viewOrderBook'
headers = {'content-type': 'application/json'}
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":5510,"1":5510},
"quantity":{"0":1.0,"1":-1.0,"traderId":{"0":1,"1":1},
"iMatched":{"0":true,"1":true,
"timeStampUTC":{"0":1547976585838,"1":1547976602766}}'
~~~~

Using a stream is only necessary if the order book is large and the request is truncated by the server, but it doesn't do any harm.

### View open trades

~~~~
url = 'https://https://blocmarket.herokuapp.com/viewOpenTrades'
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

~~~~
'{"marketId":{"12":1,"15":1,"18":1,"21":1,"24":1},
"price":{"12":4300,"15":4300,"18":4300,"21":4300,"24":4300},
"quantity":{"12":-1.0,"15":-1.0,"18":-1.0,"21":-1.0,"24":-1.0},
"traderId":{"12":1,"15":1,"18":1,"21":1,"24":1},"timeStampUTC":
{"12":1547976742476,"15":1547976750344,"18":1547976758683,"21":1547976768968,"24":1547976775537}}'
~~~~

### View matched trades

~~~~
url = 'https://blocmarket.herokuapp.com/viewMatchedTrades'
headers = {'content-type': 'application/json'}
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":4300,"1":5500},"quantity":{"0":0.0,"1":0.0},"traderId":{"0":1,"1":1}}'

~~~~

### View trader summary

A view joining order book and marketTable for a particular trader.

~~~~
url = 'https://blocmarket.herokuapp.com/viewTradeSummary'
headers = {'content-type': 'application/json'}
content = {'traderId': 2}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":8000,"1":5000},
"quantity":{"0":-1.0,"1":-1.0},"traderId":{"0":2,"1":2},
"iMatched":{"0":false,"1":true},
"timeStampUTC":{"0":1547982939316,"1":1547982938964},
"marketMin":{"0":0.0,"1":0.0},"marketMax":{"0":1e4,"1":1e4}}'
~~~~


# Python demonstration

In [1]:
import requests
import json
import numpy as np
import pandas as pd
import time
import datetime

%matplotlib inline
import matplotlib.pyplot as plt

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04


In [2]:
baseurl = 'https://blocmarket.herokuapp.com/'
# baseurl = 'http://127.0.0.1:5000/'
#baseurl = 'http://127.0.0.1:5000/'

In [3]:
# Create a new user
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [4]:
# Copy keys and traderid from response
sk = response.json()['signingKey']
vk = response.json()['verifyKey']
tId = int(response.json()['traderId'])


In [5]:
# Create a new market
mkRtId = 1
mkBrId = 1


url = baseurl + 'createMarket'
content_makemarket = {"signingKey": sk,
                        "traderId": tId, 
                        "verifyKey": vk,
                        "marketRootId": 1, 
                        "marketBranchId": mkBrId, 
                        "marketMin": 1100,
                        "marketMax":7700}
# Post market
response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)


In [6]:
# response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
# View response as json
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '1', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 1,
 'marketId': 1,
 'marketMax': 7700,
 'marketMin': 1100,
 'marketRootId': 1,
 'traderId': 1}

In [7]:
# Save market id from response
mkId = response.json()['marketId']
mkId

1

In [8]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey": sk,
                     "traderId": tId,
                     "verifyKey": vk,
                     "marketId": mkId,
                     "price": 5510,
                     "quantity":1}
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [9]:
# View response as json
response.json()

{'allChecks': "{'tradeId': 1, 'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 5510,
 'quantity': 1,
 'tradeId': 1,
 'traderId': 1}

In [10]:
# View response as a dataframe
pd.DataFrame(response.json(), index=[0])


,allChecks,checks,marketId,price,quantity,tradeId,traderId
0,"{'tradeId': 1, 'inputChk': True, 'traderIdChk'...",True,1,5510,1,1,1


In [11]:
mId = response.json()['marketId']

In [12]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [13]:
# View
response.json()

{'allChecks': "{'tradeId': 2, 'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 5510,
 'quantity': -1,
 'tradeId': 2,
 'traderId': 1}

In [14]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 3.698326826095581 seconds.
matched trade: 1 in 4.175363063812256 seconds.
matched trade: 2 in 3.173060894012451 seconds.
matched trade: 3 in 3.3302371501922607 seconds.
matched trade: 4 in 3.225843906402588 seconds.


In [15]:
content_maketrade

{'signingKey': '9c12f26a0512d196288095c22a9588c865831e29b14329d1a991a753787758e5',
 'traderId': 1,
 'verifyKey': 'b33d076ab544c04bde89e3fe491da4467e700c151c6dfb1e1e675e6524f88f69',
 'marketId': 1,
 'price': 5510,
 'quantity': 1}

In [16]:
# Create many markets
url = baseurl+'createMarket'

for i in range(10):
    t = time.time()

    content_makemarket = {"signingKey": sk,
      "traderId": int(tId), "verifyKey": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

market created: 1 in 1.9016287326812744 seconds.
market created: 2 in 2.4574122428894043 seconds.
market created: 3 in 2.765756130218506 seconds.
market created: 4 in 4.14372992515564 seconds.
market created: 5 in 5.93013596534729 seconds.
market created: 6 in 12.147929191589355 seconds.
failed to create market 7
failed to create market 8
failed to create market 9
failed to create market 10


In [17]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': False, 'traderIdChk': True, 'marketId': '7', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'False',
 'marketBranchId': 1,
 'marketId': 7,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 10,
 'traderId': 1}

In [18]:
# Create many sub-markets
url = baseurl+'createMarket'
marketRootId = tId

for i in range(10):
    t = time.time()

    content_makemarket = {"signingKey": sk,
      "traderId": int(tId), "verifyKey": vk,
      "marketRootId": marketRootId, "marketBranchId": 1+i, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create sub market ' +str(i+1))
    

failed to create sub market 1
market created: 2 in 8.654430866241455 seconds.
market created: 3 in 11.619898080825806 seconds.


KeyboardInterrupt: 

In [19]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '15', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': True, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 10,
 'marketId': 15,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 1,
 'traderId': 1}

In [7]:
# View maximum and minimum of each market
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)


In [13]:
z = pd.read_json(response.json())
z.loc[z['marketId']==1,'marketDesc'][0]



'Market root : 1, branch: 1'

In [22]:
z = pd.read_json(response.json())
max(z['marketRootId'])


6

In [25]:
# View whole order book
url = baseurl+'viewOrderBook'
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [26]:
pd.read_json(response.json())


,tradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC
0,1,1,5510,1,1,True,2019-05-11 06:09:14.182
1,2,1,5510,-1,1,True,2019-05-11 06:09:22.659
10,11,1,5510,-1,1,True,2019-05-11 06:09:30.036
11,12,1,5510,1,1,True,2019-05-11 06:09:30.123
2,3,1,5510,-1,1,True,2019-05-11 06:09:29.335
3,4,1,5510,1,1,True,2019-05-11 06:09:29.425
4,5,1,5510,-1,1,True,2019-05-11 06:09:29.515
5,6,1,5510,1,1,True,2019-05-11 06:09:29.598
6,7,1,5510,-1,1,True,2019-05-11 06:09:29.689
7,8,1,5510,1,1,True,2019-05-11 06:09:29.776


In [27]:
content_makemarket

{'signingKey': '2680fdff48ebce482bdd234a2c0f83f3eac9ae335dfde667b38f33e06777a12a',
 'traderId': 1,
 'verifyKey': '843d2fac0cb987a50f96db9002d5f08733feb937be6a581b5520a106590adb39',
 'marketRootId': 1,
 'marketBranchId': 10,
 'marketMin': 0,
 'marketMax': 0.7}

In [45]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    content_maketrade['price'] = 4300
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 0.10569000244140625 seconds.
unmatched trade: 1 in 0.09033203125 seconds.
unmatched trade: 2 in 0.09038496017456055 seconds.
unmatched trade: 3 in 0.09141302108764648 seconds.
unmatched trade: 4 in 0.0867002010345459 seconds.


In [30]:
a = response.json()
a

{'allChecks': "{'tradeId': 17, 'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 4300,
 'quantity': -1,
 'tradeId': 17,
 'traderId': 1}

In [31]:
# View all open (unmatched trades)
url = baseurl+'viewOpenTrades'

content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [32]:
pd.read_json(response.json())

,tradeId,marketId,price,quantity,traderId,timeStampUTC
12,13,1,4300,-1,1,2019-05-11 06:11:06.429
13,14,1,4300,-1,1,2019-05-11 06:11:06.519
14,15,1,4300,-1,1,2019-05-11 06:11:06.611
15,16,1,4300,-1,1,2019-05-11 06:11:06.701
16,17,1,4300,-1,1,2019-05-11 06:11:06.791


In [33]:
# View matched trades (summed over marketId/price/traderId)
url = baseurl+'viewMatchedTrades'
content = {'marketId': mkId}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [34]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,5510,0,1


In [44]:
# Create many matched trades (own)
url = baseurl+'createTrade'

for i in range(5): 
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.19957184791564941 seconds.
matched trade: 1 in 0.18017983436584473 seconds.
matched trade: 2 in 0.1808781623840332 seconds.
matched trade: 3 in 0.18397116661071777 seconds.
matched trade: 4 in 0.17993497848510742 seconds.


In [36]:
# View summary of trader for a particular trader
url = baseurl+'viewTradeSummary'
content = {'traderId': int(1)}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [37]:
pd.read_json(response.json())

,tradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC,marketRootId,marketBranchId,marketMin,marketMax,marketMinOutcome,marketMaxOutcome
0,1,1,5510,1,1,True,2019-05-11 06:09:14.182,1,1,1100,1100,-4410,-4410
1,2,1,5510,-1,1,True,2019-05-11 06:09:22.659,1,1,1100,1100,4410,4410
10,11,1,5510,-1,1,True,2019-05-11 06:09:30.036,1,1,1100,1100,4410,4410
11,12,1,5510,1,1,True,2019-05-11 06:09:30.123,1,1,1100,1100,-4410,-4410
12,18,1,4300,-1,1,False,2019-05-11 06:11:33.461,1,1,1100,1100,3200,3200
13,13,1,4300,-1,1,True,2019-05-11 06:11:06.429,1,1,1100,1100,3200,3200
14,19,1,4300,1,1,True,2019-05-11 06:11:33.550,1,1,1100,1100,-3200,-3200
15,20,1,4300,-1,1,False,2019-05-11 06:11:33.641,1,1,1100,1100,3200,3200
16,14,1,4300,-1,1,True,2019-05-11 06:11:06.519,1,1,1100,1100,3200,3200
17,21,1,4300,1,1,True,2019-05-11 06:11:33.728,1,1,1100,1100,-3200,-3200


In [38]:
# Get a signed UTC timestamp
url = baseurl+'getSignedUTCTimestamp'
content = {}
response = requests.get(url, data=json.dumps(content), headers=headers)


In [39]:
response.json()

{'timeStampUTC': '2019-05-11 06:11:48.549415',
 'timeStampUTCSignature': 'b"\\xc5\\xa6zT)\\xf9\\x05\\xb4\\xe6\\x1cs\\x04n\'=\\xa6s\\xb9\\x8c[\\xce2\\x9bZ\\xd5\\xaf\\xb4\\xc6@\\x03G\\xd2\\x1e\\x977f;r\\xca/\\x8at\\t\\x83\\xda\\xd5\\xb8C\\xb0;\\x9a0|\\x0c83\\xf5\\r\\xf8P\\x10=\\xa4\\x07"',
 'verifyKey': 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'}

In [42]:
# View summary of trader for a particular trader
url = baseurl+'viewTickHistory'
content = {'marketId': int(1), 'startTime': datetime.datetime(2019,1,1).timestamp()*1000, 'endTime': 2e9*1000}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [43]:
pd.read_json(response.json())

,tickType,tradeId,xTradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC
0,BID,1,2.0,1,5510,1,1,True,2019-05-11 06:09:14.182
1,ASK,13,19.0,1,4300,-1,1,True,2019-05-11 06:11:06.429
2,ASK,14,19.0,1,4300,-1,1,True,2019-05-11 06:11:06.519
3,ASK,15,19.0,1,4300,-1,1,True,2019-05-11 06:11:06.611
4,ASK,16,19.0,1,4300,-1,1,True,2019-05-11 06:11:06.701
5,ASK,17,19.0,1,4300,-1,1,True,2019-05-11 06:11:06.791
6,ASK,18,NaN,1,4300,-1,1,False,2019-05-11 06:11:33.461


In [93]:
np.int64(30)


36


# Curl API examples



## Create user

~~~~
curl --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createUser
~~~~
        
## Create market

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketRootId": 1, 
"marketBranchId": 5, 
"marketMin": 0, 
"marketMax": 1e4}' 
--header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createMarket
~~~~
    
## Create trade

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketId": 1,
"price": 1212,
"quantity": 1}' --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createTrade

~~~~
        
## View order book (output to file)

~~~~
curl --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/viewOrderBook --output ob.txt
~~~~    
        
Also can load json from file with something like 
~~~~ 
curl -d "@data.json" -X POST http://blocmarket.herokuapp.com/data 
~~~~


# Matlab API examples


## Create user

~~~~
resp = webread('http://blocmarket.herokuapp.com/createUser')

~~~~

Returns something like

~~~~
'{"signingKey": '57f611e7cb7c5d4123be69663ccd06442c64db1a4f1161399dbb1b9bb6cc0e06'
           "traderId": '2'
      "verifyKey": 'ddaa5b0119a4ccfeffa36be0e6aac3682f798d021d5ded1d90512f2226e727b5'}'
~~~~

## Create market

~~~~
data = struct('signingKey', signingKey,...
'traderId', tId, ...
'verifyKey', verifyKey ,...
'marketRootId', 2, ...
'marketBranchId', 1, ...
'marketMin', 100, ...
'marketMax', 1e4)
resp = webwrite('http://blocmarket.herokuapp.com/createMarket', data, weboptions('RequestMethod', 'post','MediaType', 'application/json'))
~~~~

Returns something like

~~~~
'{
             "checks": 'True'
     "marketBranchId": 1
          "marketMax": 10000
          "marketMin": 100
       "marketRootId": 2
           "traderId": 1
}'
~~~~
## Create trade

~~~~
data = struct('signingKey', signingKey,...
'traderId', tId,...
'verifyKey', verifyKey ,...
'marketId', 1,...
'price', 5100,...
'quantity', 1)
resp = webwrite('http://blocmarket.herokuapp.com/createTrade', data, weboptions('MediaType', 'application/json'))
~~~~

Returns something like
~~~~
'{
       "checks": 'True'
     "marketId": 1
        "price": 5100
     "quantity": 1
     "traderId": 1
 '}S
~~~~

## View market bounds
~~~~
data = struct('marketId', 1)
resp = webwrite('http://blocmarket.herokuapp.com/viewMarketBounds', data, weboptions('RequestMethod', 'post','contentType', 'json', 'Timeout', 10))
~~~~

## View order book
~~~~
data = struct('marketId':1)
resp = webwrite('http://blocmarket.herokuapp.com/viewOrderBook', data, weboptions('RequestMethod', 'post','contentType', 'json'), 'Timeout', 10)
~~~~

## View trade summary

~~~~
data = struct('traderId',1);
resp = webwrite('http://blocmarket.herokuapp.com/viewTradeSummary', data, weboptions('RequestMethod', 'post','MediaType', 'application/json', 'Timeout', 10))
~~~~


# Powershell

TODO


    

test cases with mismatched traderId/verifyKey
views to specific market
javascript/powershell for api
enforce max decimals DONE
table based checkcollateral
put up on digitalocean

write up another layer of trading interface